In [3]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import*

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, DoubleType

import secrets

In [4]:
# 2. Initializing SparkSession

spark = SparkSession.builder.appName("loadbranch").getOrCreate()

Extracting files 

In [5]:
# 3. Read branch-json File with Spark DataFrame API

df_branch = spark.read.load(r"C:\Users\Learner_XZHCG217\Desktop\TEK-Dataengineering\Git\Capstone\files\cdw_sapp_branch.json",\
     format="json", header = True,inferSchema = True)

# df_customer = spark.read.json(r"C:\Users\Learner_XZHCG217\Desktop\TEK-Dataengineering\Git\Capstone\files\cdw_sapp_custmer.json")

df_branch.printSchema()

root
 |-- BRANCH_CITY: string (nullable = true)
 |-- BRANCH_CODE: long (nullable = true)
 |-- BRANCH_NAME: string (nullable = true)
 |-- BRANCH_PHONE: string (nullable = true)
 |-- BRANCH_STATE: string (nullable = true)
 |-- BRANCH_STREET: string (nullable = true)
 |-- BRANCH_ZIP: long (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)



In [6]:
# Change the data type of LAST_UPDATED to timestamp

df_branch = df_branch.withColumn('LAST_UPDATED', df_branch['LAST_UPDATED'].cast('timestamp'))


In [7]:
# if source value is null for BRANCH_ZIP, load (00000). else direct move

df_branch.filter(isnull('BRANCH_ZIP')).show()

# Replace 0 for null for all integer columns

df_branch.na.fill({'BRANCH_ZIP': 0000}).show()

# ----change the first row to "null" initially and verify if it was replaced with 0---.

+-----------+-----------+-----------+------------+------------+-------------+----------+------------+
|BRANCH_CITY|BRANCH_CODE|BRANCH_NAME|BRANCH_PHONE|BRANCH_STATE|BRANCH_STREET|BRANCH_ZIP|LAST_UPDATED|
+-----------+-----------+-----------+------------+------------+-------------+----------+------------+
+-----------+-----------+-----------+------------+------------+-------------+----------+------------+

+-----------------+-----------+------------+------------+------------+-------------------+----------+-------------------+
|      BRANCH_CITY|BRANCH_CODE| BRANCH_NAME|BRANCH_PHONE|BRANCH_STATE|      BRANCH_STREET|BRANCH_ZIP|       LAST_UPDATED|
+-----------------+-----------+------------+------------+------------+-------------------+----------+-------------------+
|        Lakeville|          1|Example Bank|  1234565276|          MN|       Bridle Court|     55044|2018-04-18 16:51:47|
|          Huntley|          2|Example Bank|  1234618993|          IL|  Washington Street|     60142|20

In [8]:
# change phone number (XXX)XXX-XXXX   --- phone number has 10 digits

df_branch = df_branch.withColumn('BRANCH_PHONE', concat(lit('('), df_branch['BRANCH_PHONE'].substr(1,3), lit(')'), \
        df_branch['BRANCH_PHONE'].substr(4, 3), lit('-'), df_branch['BRANCH_PHONE'].substr(7, 6)))

#df_customer['CUST_PHONE'].show(5)
df_branch.show(5)

# if you run the cell twice.... the phone number changes to all 5's

+-----------------+-----------+------------+-------------+------------+-----------------+----------+-------------------+
|      BRANCH_CITY|BRANCH_CODE| BRANCH_NAME| BRANCH_PHONE|BRANCH_STATE|    BRANCH_STREET|BRANCH_ZIP|       LAST_UPDATED|
+-----------------+-----------+------------+-------------+------------+-----------------+----------+-------------------+
|        Lakeville|          1|Example Bank|(123)456-5276|          MN|     Bridle Court|     55044|2018-04-18 16:51:47|
|          Huntley|          2|Example Bank|(123)461-8993|          IL|Washington Street|     60142|2018-04-18 16:51:47|
|SouthRichmondHill|          3|Example Bank|(123)498-5926|          NY|    Warren Street|     11419|2018-04-18 16:51:47|
|       Middleburg|          4|Example Bank|(123)466-3064|          FL| Cleveland Street|     32068|2018-04-18 16:51:47|
|    KingOfPrussia|          5|Example Bank|(123)484-9701|          PA|      14th Street|     19406|2018-04-18 16:51:47|
+-----------------+-----------+-

In [9]:
# Verify the target data types

df_branch.printSchema()

root
 |-- BRANCH_CITY: string (nullable = true)
 |-- BRANCH_CODE: long (nullable = true)
 |-- BRANCH_NAME: string (nullable = true)
 |-- BRANCH_PHONE: string (nullable = true)
 |-- BRANCH_STATE: string (nullable = true)
 |-- BRANCH_STREET: string (nullable = true)
 |-- BRANCH_ZIP: long (nullable = true)
 |-- LAST_UPDATED: timestamp (nullable = true)



LOADING

In [12]:
df_branch.write.format("jdbc")\
                .mode("append")\
                .option("url","jdbc:mysql://localhost:3306/creditcard_capstone")\
                .option("dbtable","CDW_SAPP_BRANCH")\
                .option("user","root")\
                .option("password","password").save()

In [13]:
# read from the table

df_new1 = spark.read.format("jdbc") \
  .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
  .option("dbtable", "CDW_SAPP_BRANCH") \
  .option("user","root") \
  .option("password", "password") \
  .load()

df_new1.show(10)


+-----------------+-----------+------------+-------------+------------+-----------------+----------+-------------------+
|      BRANCH_CITY|BRANCH_CODE| BRANCH_NAME| BRANCH_PHONE|BRANCH_STATE|    BRANCH_STREET|BRANCH_ZIP|       LAST_UPDATED|
+-----------------+-----------+------------+-------------+------------+-----------------+----------+-------------------+
|        Lakeville|          1|Example Bank|(123)456-5276|          MN|     Bridle Court|     55044|2018-04-18 16:51:47|
|          Huntley|          2|Example Bank|(123)461-8993|          IL|Washington Street|     60142|2018-04-18 16:51:47|
|SouthRichmondHill|          3|Example Bank|(123)498-5926|          NY|    Warren Street|     11419|2018-04-18 16:51:47|
|       Middleburg|          4|Example Bank|(123)466-3064|          FL| Cleveland Street|     32068|2018-04-18 16:51:47|
|    KingOfPrussia|          5|Example Bank|(123)484-9701|          PA|      14th Street|     19406|2018-04-18 16:51:47|
|         Paterson|          7|E